# CSE 252B: Computer Vision II, Winter 2019 – Assignment 3
### Instructor: Ben Ochoa
### Due: Wednesday, February 20, 2019, 11:59 PM

## Instructions
* Review the academic integrity and collaboration policies on the course website.
* This assignment must be completed individually.
* This assignment contains both math and programming problems.
* All solutions must be written in this notebook
* Math problems must be done in Markdown/LATEX. Remember to show work and describe your solution.
* Programming aspects of this assignment must be completed using Python in this notebook.
* Your code should be well written with sufficient comments to understand, but there is no need to write extra markdown to describe your solution if it is not explictly asked for.
* This notebook contains skeleton code, which should not be modified (This is important for standardization to facilate effeciant grading).
* You may use python packages for basic linear algebra, but you may not use packages that directly solve the problem. Ask the instructor if in doubt.
* You must submit this notebook exported as a pdf. You must also submit this notebook as an .ipynb file.
* Your code and results should remain inline in the pdf (Do not move your code to an appendix).
* You must submit both files (.pdf and .ipynb) on Gradescope. You must mark each problem on Gradescope in the pdf.
* It is highly recommended that you begin working on this assignment early.

## Problem 1 (Programming):  Estimation of the Camera Pose - Outlier rejection (20 points)
  Download input data from the course website.  The file
  hw3_points3D.txt contains the coordinates of 60 scene points
  in 3D (each line of the file gives the $\tilde{X}_i$, $\tilde{Y}_i$,
  and $\tilde{Z}_i$ inhomogeneous coordinates of a point).  The file
  hw3_points2D.txt contains the coordinates of the 60
  corresponding image points in 2D (each line of the file gives the
  $\tilde{x}_i$ and $\tilde{y}_i$ inhomogeneous coordinates of a
  point).  The corresponding 3D scene and 2D image points contain both
  inlier and outlier correspondences.  For the inlier correspondences,
  the scene points have been randomly generated and projected to image
  points under a camera projection matrix (i.e., $\boldsymbol{x}_i = \boldsymbol{P}
  \boldsymbol{X}_i$), then noise has been added to the image point
  coordinates.

  The camera calibration matrix was calculated for a $1280 \times 720$
  sensor and $45\,^\circ$ horizontal field of view lens.  The
  resulting camera calibration matrix is given by
  
  $\boldsymbol{K} = \left[
    \begin{array}{c c c}
      1545.0966799187809 & 0 & 639.5\\
      0 & 1545.0966799187809 & 359.5\\
      0 & 0 & 1
    \end{array}\right]$
    
  For each image point $\boldsymbol{x} = (x, y, w)^\top = (\tilde{x},
  \tilde{y}, 1)^\top$, calculate the point in normalized coordinates
  $\hat{\boldsymbol{x}} = \boldsymbol{K}^{-1} \boldsymbol{x}$.

  Determine the set of inlier point correspondences using the
  M-estimator Sample Consensus (MSAC) algorithm, where the maximum
  number of attempts to find a consensus set is determined adaptively.
  For each trial, use the 3-point algorithm of Finsterwalder (as
  described in the paper by Haralick et al.) to estimate the camera
  pose (i.e., the rotation $\boldsymbol{R}$ and translation $\boldsymbol{t}$ from the
  world coordinate frame to the camera coordinate frame), resulting in
  up to 4 solutions, and calculate the error and cost for each
  solution.  Note that the 3-point algorithm requires the 2D points in
  normalized coordinates, not in image coordinates.  Calculate the
  projection error, which is the (squared) distance between projected
  points (the points in 3D projected under the normalized camera
  projection matrix $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$) and the
  measured points in normalized coordinates (hint: the error tolerance
  is simpler to calculate in image coordinates using $\boldsymbol{P} =
  \boldsymbol{K} [\boldsymbol{R} | \boldsymbol{t}]$ than in normalized coordinates using
  $\hat{\boldsymbol{P}} = [\boldsymbol{R} | \boldsymbol{t}]$).

  
  Hint: this problem has codimension 2. 
 

#### Report your values for:
 * the probability $p$ that as least one of the random samples does not contain any outliers
 * the probability $\alpha$ that a given point is an inlier
 * the resulting number of inliers
 * the number of attempts to find the consensus set

In [379]:
import numpy as np
import time

def Homogenize(x):
    # converts points from inhomogeneous to homogeneous coordinates
    return np.vstack((x,np.ones((1,x.shape[1]))))

def Dehomogenize(x):
    # converts points from homogeneous to inhomogeneous coordinates
    return x[:-1]/x[-1]

    
# load data
x0=np.loadtxt('hw3_points2D.txt').T
X0=np.loadtxt('hw3_points3D.txt').T
print('x is', x0.shape)
print('X is', X0.shape)

K = np.array([[1545.0966799187809, 0, 639.5], 
      [0, 1545.0966799187809, 359.5], 
      [0, 0, 1]])

print('K =')
print(K)
    
def ComputeCost(P, x, X, K):
    # Inputs:
    #    P - camera projection matrix
    #    x - 2D groundtruth image points
    #    X - 3D groundtruth scene points
    #    K - camera calibration matrix
    #
    # Output:
    #    cost - total projection error
    n = x.shape[1]
    covarx = np.eye(2*n) # covariance propagation
    
    """your code here"""
    x = Homogenize(x)
    P = np.matmul(K,P)
    xx = Homogenize(projection(P,X))  # projection
    cost = sum([np.linalg.norm(xx[:, i] - x[:,i]) for i in range(n)]) # SSE
    return cost

def reprojection_error(P,x,X,K):
    # x-normalized 2D coordinates
    P = np.matmul(K,P)
    x_proj = projection(P,X)
    x_denormalize = Dehomogenize(K.dot(x))
    error = np.power(np.linalg.norm((x_denormalize-x_proj),axis=0),2)
    return error

def projection(P,X):
    # projects 3d points X to 2d using projection matrix P
    return Dehomogenize(np.matmul(P,Homogenize(X)))

def length(X,samples):
    a = np.sqrt((X[0,samples[1]]-X[0,samples[2]])**2+\
                (X[1,samples[1]]-X[1,samples[2]])**2+\
                (X[2,samples[1]]-X[2,samples[2]])**2)
    b = np.sqrt((X[0,samples[0]]-X[0,samples[2]])**2+\
                (X[1,samples[0]]-X[1,samples[2]])**2+\
                (X[2,samples[0]]-X[2,samples[2]])**2)
    c = np.sqrt((X[0,samples[0]]-X[0,samples[1]])**2+\
                (X[1,samples[0]]-X[1,samples[1]])**2+\
                (X[2,samples[0]]-X[2,samples[1]])**2)
    return a,b,c

def unit_vec(x,samples):
    unit_vecs = []
    for i in range(3):
        j = np.sign(x[2,samples[i]])*x[:,samples[i]].reshape(3,1)/\
        np.linalg.norm(x[:,samples[i]])
        unit_vecs.append(j)
    cos_alpha = np.dot(unit_vecs[1].T,unit_vecs[2])
    cos_beta = np.dot(unit_vecs[0].T,unit_vecs[2])
    cos_gamma = np.dot(unit_vecs[0].T,unit_vecs[1])
    return unit_vecs,cos_alpha,cos_beta,cos_gamma 

def Finster(x,X,samples):
    result = []
    a,b,c = length(X,samples)
    unit_vecs,cos_alpha,cos_beta,cos_gamma = unit_vec(x,samples)
    sin_alpha_square = 1 - cos_alpha**2
    sin_beta_square = 1 - cos_beta**2
    sin_gamma_square = 1 - cos_gamma**2
    
    G = c**2*(c**2*sin_beta_square-b**2*sin_gamma_square)
    H = b**2*(b**2-a**2)*sin_gamma_square+\
    c**2*(c**2+2*(a**2))*sin_beta_square+\
    2*(b**2)*(c**2)*(-1+cos_alpha*cos_beta*cos_gamma)
    I = b**2*(b**2-c**2)*sin_alpha_square+\
    a**2*(a**2+2*(c**2))*sin_beta_square+\
    2*(a**2)*(b**2)*(-1+cos_alpha*cos_beta*cos_gamma)
    J = a**2*(a**2*sin_beta_square-b**2*sin_alpha_square)
    lambda_value = np.roots(np.array([G,H,I,J]).reshape(4))
    lambda_value = np.real(lambda_value[np.isreal(lambda_value)])
    for lambda_0 in lambda_value:
        A = 1+lambda_0
        B = -cos_alpha
        C = ((b**2-a**2)-lambda_0*(c**2))/float(b**2)
        D = -lambda_0*cos_gamma
        E = (a**2+lambda_0*(c**2))/float(b**2)*cos_beta
        F = -a**2/float(b**2)+lambda_0*((b**2-c**2)/float(b**2))
        
        if B**2-A*C>=0:
            p = np.sqrt(B**2-A*C)
            q = np.sign(B*E-C*D)*np.sqrt(E**2-C*F)

            m = (-B+p)/float(C)
            n = (-(E-q))/float(C)

            A = b**2 - m**2*(c**2)
            B = c**2*(cos_beta-n)*m-b**2*cos_gamma
            C = -c**2*(n**2)+2*(c**2)*n*cos_beta+b**2-c**2

            if B**2-A*C>=0:
                u_large = -np.sign(B)/A*(np.fabs(B.reshape(1))+np.sqrt(B**2-A*C))
                v = u_large*m+n
                if a**2/(u_large**2+v**2-2*u_large*v*cos_alpha)>=0:
                    s_1 = np.sqrt(a**2/(u_large**2+v**2-2*u_large*v*cos_alpha))
                    result.append((float(s_1),float(u_large),float(v)))
                u_small = C/(A*u_large)
                v = u_small*m+n
                if a**2/(u_small**2+v**2-2*u_large*v*cos_alpha)>=0:
                    s_1 = np.sqrt(a**2/(u_small**2+v**2-2*u_large*v*cos_alpha))
                    result.append((float(s_1),float(u_small),float(v)))

            m = (-B-p)/C
            n = (-(E+q))/C

            A = b**2 - m**2*(c**2)
            B = c**2*(cos_beta-n)*m-b**2*cos_gamma
            C = -c**2*(n**2)+2*(c**2)*n*cos_beta+b**2-c**2

            if B**2-A*C>=0:
                u_large = -np.sign(B)/A*(np.fabs(B.reshape(1))+np.sqrt(B**2-A*C))
                v = u_large*m+n
                if a**2/(u_large**2+v**2-2*u_large*v*cos_alpha)>=0:
                    s_1 = np.sqrt(a**2/(u_large**2+v**2-2*u_large*v*cos_alpha))
                    result.append((float(s_1),float(u_large),float(v)))
                u_small = C/(A*u_large)
                v = u_small*m+n
                if a**2/(u_small**2+v**2-2*u_large*v*cos_alpha)>=0:
                    s_1 = np.sqrt(a**2/(u_small**2+v**2-2*u_large*v*cos_alpha))
                    result.append((float(s_1),float(u_small),float(v)))
    
    return result,unit_vecs

def camera_coordinate(candidates,unit_vecs):
    coordinates = []
    for s_1,u,v in candidates:
        p_1 = s_1*unit_vecs[0]
        p_2 = u*s_1*unit_vecs[1]
        p_3 = v*s_1*unit_vecs[2]
        coordinates.append((p_1,p_2,p_3))
    return coordinates

def world_coordinate(samples):
    coordinates = []
    for index in samples:
        coordinates.append(X0[:,index].reshape(3,1))
    return coordinates

def LeastSquare_estimation(A,B):
    # B = RA+t
    mu_x = sum(A)/float(len(A))
    mu_y = sum(B)/float(len(B))
    sigma_xy = np.zeros((3,3))
    for i in range(len(A)):
        sigma_xy = sigma_xy + (B[i]-mu_y).dot((A[i]-mu_x).T)
    sigma_xy = sigma_xy/float(len(A))
    
    U, D, VT = np.linalg.svd(sigma_xy, full_matrices=True)
    S = np.eye(3)
    if np.linalg.det(U)*np.linalg.det(VT.T)<0:
        S[2,2] = -1.
    
    R = U.dot(S).dot(VT)
    t = mu_y - R.dot(mu_x)
    return R,t

def consensus_cost(P,x,X,error,tol):
    cost = 0.
    num_inlier = 0.
    inliers = []
    for i in range(x.shape[1]):
        if error[i]<=tol:
            cost = cost+error[i]
            num_inlier = num_inlier+1
            inliers.append(i)
        else:
            cost = cost+tol
    return cost,num_inlier,inliers


x is (2, 60)
X is (3, 60)
K =
[[1.54509668e+03 0.00000000e+00 6.39500000e+02]
 [0.00000000e+00 1.54509668e+03 3.59500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [393]:
from scipy.stats import chi2
import random 

def MSAC(x, X, K, thresh, tol, p):
    # Inputs:
    #    x - 2D inhomogeneous image points
    #    X - 3D inhomogeneous scene points
    #    K - camera calibration matrix
    #    thresh - cost threshold
    #    tol - reprojection error tolerance 
    #    p - probability that as least one of the random samples does not contain any outliers   
    #
    # Output:
    #    consensus_min_cost - final cost from MSAC
    #    consensus_min_cost_model - camera projection matrix P
    #    inliers - list of indices of the inliers corresponding to input data
    #    trials - number of attempts taken to find consensus set
    
    """your code here"""
    x = np.linalg.inv(K).dot(Homogenize(x))
    trials = 0.
    max_trials = np.inf
    consensus_min_cost = np.inf
    
    while trials < max_trials :
        # randomly select 3 points
        samples = random.sample(range(60),3)    
        # calculate camera poses
        candidates_uv,unit_vecs = Finster(x,X,samples)
        camera_coord_vec =camera_coordinate(candidates_uv,unit_vecs)
        world_coord = world_coordinate(samples)

        for camera_coord in camera_coord_vec:
            # calculate model
            R,t = LeastSquare_estimation(world_coord,camera_coord)
            P = np.concatenate((R, t), axis=1)
            # calculate error
            error = reprojection_error(P,x,X, K)
            # calculate consensusCost
            cost,num_inlier,inliers = consensus_cost(P,x,X,error,tol)
            # update model
            if cost<consensus_min_cost:
                consensus_min_cost = cost
                R_best, t_best, P_best = R, t, P
                # update max_trials
                w = float(num_inlier)/float(x.shape[1])
                if w>0:
                    max_trials = np.log(1-p)/np.log(1-w**3)  
        trials = trials+1
    
    consensus_min_cost_model = np.matmul(K,P_best)
    error = reprojection_error(P_best,x,X, K)
    consensus_min_cost,num_inlier,inliers = consensus_cost(P_best,x,X,error,tol)
    return consensus_min_cost, consensus_min_cost_model, inliers, trials


# MSAC parameters 
thresh = 200
tol = 6
p = 0.99
alpha = 0.95

tic=time.time()

cost_MSAC, P_MSAC, inliers, trials = MSAC(x0, X0, K, thresh, tol, p)

# choose just the inliers
x = x0[:,inliers]
X = X0[:,inliers]

toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
print('%d iterations'%trials)
print('inlier count: ',len(inliers))
print('MSAC Cost=%.9f'%cost_MSAC)
print('P = ')
print(P_MSAC)
print('inliers: ',inliers)


took 0.016980 secs
18 iterations
inlier count:  45
MSAC Cost=181.823066651
P = 
[[ 8.72842205e+02 -6.68553712e+02  1.25994699e+03  1.20899074e+05]
 [ 1.27449829e+03 -3.40429607e+02 -8.81093519e+02  7.51232357e+04]
 [ 6.95378385e-01  6.23796352e-01  3.56829110e-01  1.75856845e+02]]
inliers:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 39, 40, 41, 42, 43, 44, 45, 46, 47, 49]


#### Final values for parameters
* $p$ =0.99
* $\alpha$ =0.95
* tolerance = 6
* num_inliers = 45
* num_attempts = 18

## Problem 2 (Programming): Estimation of the Camera Pose - Linear Estimate (30 points)
  Estimate the normalized camera projection matrix
  $\hat{\boldsymbol{P}}_\text{linear} = [\boldsymbol{R}_\text{linear} |
  \boldsymbol{t}_\text{linear}]$ from the resulting set of inlier
  correspondences using the linear estimation method (based on the
  EPnP method) described in lecture. Report the resulting 
  $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$.

In [394]:
def coef(X,C_1,VT,s):
    # X-inhomogeneous 3D points
    A = s*VT.T
    b = X-C_1
    x = np.linalg.solve(A,b)
    return (1-x[0]-x[1]-x[2],x[0],x[1],x[2])

def aux_mat(x,X,C_1,VT,s):
    x = Dehomogenize(x)
    A = np.zeros((1,12))
    for i in range(X.shape[1]):
        Ai = np.zeros((2,12))
        alpha = coef(X[:,i].reshape(3,1),C_1,VT,s)
        Ai[0,0] = alpha[0]
        Ai[1,1] = alpha[0]
        Ai[0,3] = alpha[1]
        Ai[1,4] = alpha[1]
        Ai[0,6] = alpha[2]
        Ai[1,7] = alpha[2]
        Ai[0,9] = alpha[3]
        Ai[1,10] = alpha[3]
        Ai[0,2] = -alpha[0]*x[0,i]
        Ai[1,2] = -alpha[0]*x[1,i]
        Ai[0,5] = -alpha[1]*x[0,i]
        Ai[1,5] = -alpha[1]*x[1,i]
        Ai[0,8] = -alpha[2]*x[0,i]
        Ai[1,8] = -alpha[2]*x[1,i]
        Ai[0,11] = -alpha[3]*x[0,i]
        Ai[1,11] = -alpha[3]*x[1,i]
        A = np.vstack((A,Ai))
    return A[1:,:]

def ComputeCameraControl(x,X,C_1,VT,s):
    control_pts_vec = []
    A = aux_mat(x,X,C_1,VT,s) # left matrix
    # right null space
    U, D, VT = np.linalg.svd(A, full_matrices=False)
    control_pts_vec.append(VT[11,0:3].reshape(3,1))
    control_pts_vec.append(VT[11,3:6].reshape(3,1))
    control_pts_vec.append(VT[11,6:9].reshape(3,1))
    control_pts_vec.append(VT[11,9:12].reshape(3,1))
    return control_pts_vec

def getCemeraCoord(X,control_pts_cam_vec,C_1,VT,s,sigma):
    X_cam_vec = []
    # Deparameterize 3D points
    for i in range(X.shape[1]):
        alpha = coef(X[:,i].reshape(3,1),C_1,VT,s)
        X_cam = np.zeros((3,1))
        for j in range(4):
            X_cam = X_cam + alpha[j]*control_pts_cam_vec[j]
        X_cam_vec.append(X_cam)
    
    # scale 3D points in camera coordinate frame
    
    X_mean_cam = sum(X_cam_vec)/len(X_cam_vec)
    cov_X = np.zeros((3,3))
    for i in range(len(X_cam_vec)):
        cov_X = cov_X + \
            (X_cam_vec[i]-X_mean_cam).dot((X_cam_vec[i]-X_mean_cam).T)
    cov_X = cov_X/len(X_cam_vec)
    sigma_cam = np.trace(cov_X)
    if X_mean_cam[2]<0:
        beta = -np.sqrt(sigma/sigma_cam)
    else:
        beta = np.sqrt(sigma/sigma_cam)
    
    return [beta*v for v in X_cam_vec]


def EPnP(x, X, K):
    # Inputs:
    #    x - 2D inlier points
    #    X - 3D inlier points
    # Output:
    #    P - normalized camera projection matrix
    
    """your code here"""
    x_origin = x
    x = np.linalg.inv(K).dot(Homogenize(x))
    
    control_pts_world_vec = []
    X_mean = np.mean(X,axis=1).reshape(3,1)
    X_covar = np.zeros((3,3))
    for i in range(X.shape[1]):
        X_covar+=(X[:,i].reshape(3,1)-X_mean).dot((X[:,i].reshape(3,1)-X_mean).T)
    X_covar /= X.shape[1]
    U, D, VT = np.linalg.svd(X_covar)
    sigma = sum(D)
    s = np.sqrt(sigma/3.0)

    control_pts_world_vec.append(X_mean)
    control_pts_world_vec.append(s*VT[0,:].reshape(3,1)+X_mean)
    control_pts_world_vec.append(s*VT[1,:].reshape(3,1)+X_mean)
    control_pts_world_vec.append(s*VT[2,:].reshape(3,1)+X_mean)
    
    control_pts_cam_vec = ComputeCameraControl(x,X,X_mean,VT,s)
    X_cam_vec = getCemeraCoord(X,control_pts_cam_vec,X_mean,VT,s,sigma)
    
    # flat X from matrix to vector
    X_vec = [X[:,i].reshape(3,1) for i in range(X.shape[1])]
    # Linear estimation of normalized projection matrix
    R,t = LeastSquare_estimation(X_vec,X_cam_vec)
    P = np.concatenate((R, t), axis=1)
    return P

tic=time.time()
P_linear = EPnP(x, X, K)
toc=time.time()
time_total=toc-tic

# display the results
print('took %f secs'%time_total)
print('R_linear = ')
print(P_linear[:,0:3])
print('t_linear = ')
print(P_linear[:,-1])

47.175193724054616
took 0.004986 secs
R_linear = 
[[ 0.2781332  -0.69100825  0.66719527]
 [ 0.66092898 -0.36635274 -0.65494927]
 [ 0.69700416  0.62313183  0.35481252]]
t_linear = 
[  5.5477011    7.5000041  175.91864403]


## Problem 3 (Programming): Estimation of the Camera Pose - Nonlinear Estimate (30 points) 
  Use $\boldsymbol{R}_\text{linear}$ and $\boldsymbol{t}_\text{linear}$ as an
  initial estimate to an iterative estimation method, specifically the
  Levenberg-Marquardt algorithm, to determine the Maximum Likelihood
  estimate of the camera pose that minimizes the projection error
  under the normalized camera projection matrix $\hat{\boldsymbol{P}} =
  [\boldsymbol{R} | \boldsymbol{t}]$.  You must parameterize the camera rotation
  using the angle-axis representation $\boldsymbol{\omega}$ (where
  $[\boldsymbol{\omega}]_\times = \ln \boldsymbol{R}$) of a 3D rotation, which is
  a 3-vector.
  
  
  Report the initial cost (i.e. cost at iteration 0) and the cost at the end
  of each successive iteration. Show the numerical values for the final 
  estimate of the camera rotation $\boldsymbol{\omega}_\text{LM}$ and 
  $\boldsymbol{R}_\text{LM}$, and the camera translation $\boldsymbol{t}_\text{LM}$.

In [395]:
from scipy.linalg import block_diag

# Note that np.sinc is different than defined in class
def Sinc(x):
    """your code here"""
    if x==0: 
        y=1
    else: 
        y = np.sin(x)/x
    return y

def dSinc(x):
    # Returns a scalar valued sinc value
    if x==0: y=0
    else: y = np.cos(x)/x - np.sin(x)/(x**2)
    return y

def skew(w):
    # Returns the skew-symmetrix represenation of a vector
    """your code here"""
    w_skew = np.zeros((3,3))
    w_skew[0,1] = -w[2,0]
    w_skew[1,0] = w[2,0]
    w_skew[0,2] = w[1,0]
    w_skew[2,0] = -w[1,0]
    w_skew[1,2] = -w[0,0]
    w_skew[2,1] = w[0,0]    
    return w_skew

def Parameterize(R):
    # Parameterizes rotation matrix into its axis-angle representation
    """your code here"""
    cos = (np.trace(R)-1.0)/2.0
    
    v = np.ones((3,1))
    v[0,0] = R[2,1]-R[1,2]
    v[1,0] = R[0,2]-R[2,0]
    v[2,0] = R[1,0]-R[0,1]
    
    U, D, VT = np.linalg.svd(R-np.eye(3))
    u = VT[2,:].reshape(3,1)
    sin = u.T.dot(v)/2.0
    
    theta = np.arctan2(sin,cos)
    w = theta[0,0]*u/np.linalg.norm(u)
    return w, theta


def Deparameterize(w):
    # Deparameterizes to get rotation matrix
    """your code here"""
    theta = np.linalg.norm(w)
    w_skew = skew(w)
    R = np.cos(theta)*np.eye(3)+Sinc(theta)*w_skew\
            +(1-np.cos(theta))/(theta**2)*w.dot(w.T)
    return R

def Jacobian(R, w, t, X):
    # compute the jacobian matrix
    # Inputs:
    #    R - 3x3 rotation matrix
    #    w - 3x1 axis-angle parameterization of R
    #    t - 3x1 translation vector
    #    X - 3D inlier points
    #
    # Output:
    #    J - Jacobian matrix of size 2*nx6
    
    """your code here"""
    P = np.concatenate((R, t), axis=1)
    x = projection(P,X)
    X_rotated = R.dot(X)
    #x/X_rotate
    w_0 = X_rotated[2,0]+t[2,0]
    derivative_1 = np.zeros((2,3))
    derivative_1[0,0] = 1.0/w_0
    derivative_1[1,1] = 1.0/w_0
    derivative_1[0,2] = -1.0*x[0,0]/w_0
    derivative_1[1,2] = -1.0*x[1,0]/w_0
    
    #X_rotate/w
    theta = np.linalg.norm(w)
    if theta<0.00001:
        derivative_2 = skew(-X)
    else:
        s = (1-np.cos(theta))/(theta**2)
        ds = (theta*np.sin(theta)-2*(1-np.cos(theta)))/(theta**3)
        derivative_2 = Sinc(theta)*skew(-X)+\
        dSinc(theta)*np.cross(w.T,X.T).reshape(3,1).dot(1.0/theta*w.T)\
        +ds*np.cross(w.T,np.cross(w.T,X.T)).reshape(3,1).dot(1.0/theta*w.T)\
        +s*np.eye(3).dot(skew(w).dot(skew(-X))+\
                         skew(-np.cross(w.T,X.T).reshape(3,1)))
    
    #x/w
    dx_w = derivative_1.dot(derivative_2)
    
    J = np.hstack((dx_w,derivative_1))
    return J

def project_error(x,X,w,t,K):
    R = Deparameterize(w)
    P = np.concatenate((R, t), axis=1)
    x_proj = Dehomogenize(P.dot(Homogenize(X)))
    x_dehomo = Dehomogenize(x)
    error = x_dehomo - x_proj
    return error

def corvax_norm(cor, K):
    J = np.mat(np.linalg.inv(K)[:2,:2])
    return J*cor*np.linalg.inv(J)

def LM_cost(x,X,w,t,K,sigma):
    error = project_error(x,X,w,t,K)
    cost = 0
    for i in range(x.shape[1]):
        cost+=error[:,i].reshape(2,1).T.dot(np.linalg.inv(sigma)).dot(error[:,i].reshape(2,1))
    return cost

In [403]:
def LM(P, x, X, K, max_iters, lam):
    # Inputs:
    #    P - initial estimate of camera pose
    #    x - 2D inliers
    #    X - 3D inliers
    #    K - camera calibration matrix 
    #    max_iters - maximum number of iterations
    #    lam - lambda parameter
    #
    # Output:
    #    P - Final camera pose obtained after convergence

    """your code here"""
    R, t = P[:,0:3], P[:,-1]
    t = np.mat(t).T
    w,_ = Parameterize(R)
    K_inv = np.linalg.inv(K)
    x_norm = K_inv.dot(Homogenize(x))
    U = np.zeros((6,6))
    s = K_inv[0,0]**2
    x_cov = s*np.eye(2)
    x_cov_norm = corvax_norm(x_cov, K)
    epsilon = project_error(x,X,w,t,K)

    e = np.zeros((6,1))
    for i in range(x.shape[1]):
        A = Jacobian(R,w,t,X[:,i].reshape(3,1))
        U = U+A.T.dot(np.linalg.inv(x_cov)).dot(A)
        e = e+A.T.dot(np.linalg.inv(x_cov)).dot(epsilon[:,i].reshape(2,1))
    
    cost = LM_cost(x_norm,X,w,t,K,x_cov_norm)
    print('iter %03d Cost %.9f'%(0, cost))
    I = np.eye(6)
    flag=True
    for i in range(max_iters):
        if flag==False: break
        S = U + lam*I
        delta = np.dot(np.linalg.inv(S),e)
        w_temp = w+delta[0:3,0].reshape(3,1)
        t_temp = t+delta[3:,0].reshape(3,1)
        R = Deparameterize(w_temp)
        P = np.concatenate((R, t_temp), axis=1)
        cost_temp = LM_cost(x_norm,X,w_temp,t_temp,K,x_cov_norm)
    
        while cost_temp>=cost:
            lam = 10.*lam
            S = U + lam*I
            delta = np.dot(np.linalg.inv(S),e)
            w_temp = w+delta[0:3,0].reshape(3,1)
            t_temp = t+delta[3:,0].reshape(3,1)
            R = Deparameterize(w_temp)
            P = np.concatenate((R, t_temp), axis=1)
            cost_temp = LM_cost(x_norm,X,w_temp,t_temp,K,x_cov_norm)
        w = w_temp
        t = t_temp
        lam = 0.1*lam
        if abs(cost_temp-cost)<0.0000001: flag=False
        cost = cost_temp
        print('iter %03d Cost %.9f'%(i+1, cost))  

    R = Deparameterize(w)
    P = np.concatenate((R, t), axis=1)
    return P
      

# LM hyperparameters
lam = .001
max_iters = 100

tic = time.time()
P_LM = LM(P_linear, x, X, K, max_iters, lam)
w_LM,_ = Parameterize(P_LM[:,0:3])
toc = time.time()
time_total = toc-tic

# display the results
print('took %f secs'%time_total)
print('w_LM = ')
print(w_LM)
print('R_LM = ')
print(P_LM[:,0:3])
print('t_LM = ')
print(P_LM[:,-1])

iter 000 Cost 57.153259773
iter 001 Cost 57.153259466
iter 002 Cost 57.153259232
iter 003 Cost 57.153259071
iter 004 Cost 57.153258982
took 0.046836 secs
w_LM = 
[[ 1.33686787]
 [-0.03118006]
 [ 1.41412114]]
R_LM = 
[[ 0.27813314 -0.69100827  0.66719527]
 [ 0.66092894 -0.36635281 -0.65494928]
 [ 0.69700423  0.62313176  0.35481251]]
t_LM = 
[[  5.5477011 ]
 [  7.5000041 ]
 [175.91864403]]
